In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from tqdm import tqdm
import re
from difflib import SequenceMatcher
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns",1000)

In [2]:
df_events_roles = pd.read_pickle("C:/Users/Anuj/Desktop/Work/Football/data/events/events_v2.pkl")

In [3]:
df_defence_footed = pd.read_pickle("C:/Users/Anuj/Desktop/Work/Football/data/matches/match+def_lineup+footedness_ver2.pkl")

In [4]:
footedness_patterns = df_defence_footed["footedness"].unique()

In [5]:
df_players = pd.read_pickle("C:/Users/Anuj/Desktop/Work/Football/data/players.pkl")

In [9]:
df_events['playerName'] = df_events["playerName"].apply(lambda x: unidecode(x))
df_events['playerName'] = df_events["playerName"].apply(lambda x: x.replace(' ',''))
df_events['playerName'] = df_events["playerName"].apply(lambda x: x.replace('-',''))

In [10]:
df_events.loc[df_events['playerName'].str.contains("Las")]

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,wyId,foot,playerName
9123,7,Touch,[Interception],8833,"[[93.60000000000001, 34.68], [79.04, 31.96]]",2499725,Others on the ball,1613,1H,529.154121,72,178442596,8833.0,right,JamalLascelles
9203,7,Clearance,"[Interception, Not accurate]",8833,"[[91.52000000000001, 25.840000000000003], [104...",2499725,Others on the ball,1613,1H,729.699428,71,178442657,8833.0,right,JamalLascelles
9245,8,Head pass,"[Interception, Not accurate]",8833,"[[83.2, 29.240000000000002], [76.9600000000000...",2499725,Pass,1613,1H,979.855395,82,178442717,8833.0,right,JamalLascelles
9332,1,Air duel,"[Lost, Not accurate]",8833,"[[95.68, 31.28], [98.8, 33.32]]",2499725,Duel,1613,1H,1284.256499,10,178443149,8833.0,right,JamalLascelles
9334,7,Touch,[Interception],8833,"[[97.76, 31.28], [102.96000000000001, 33.32]]",2499725,Others on the ball,1613,1H,1287.359459,72,178443148,8833.0,right,JamalLascelles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591890,1,Ground attacking duel,"[Won, Accurate]",399564,"[[44.72, 9.520000000000001], [47.84, 3.4000000...",2500096,Duel,1639,2H,1275.376288,11,251668153,399564.0,right,LasseSorensen
591892,8,Simple pass,[Not accurate],399564,"[[47.84, 3.4000000000000004], [55.120000000000...",2500096,Pass,1639,2H,1276.749933,85,251668154,399564.0,right,LasseSorensen
591959,8,Simple pass,[Accurate],399564,"[[27.04, 25.16], [42.64, 19.040000000000003]]",2500096,Pass,1639,2H,1519.322589,85,251668174,399564.0,right,LasseSorensen
592009,7,Touch,[Missed ball],399564,"[[30.16, 32.64], [29.12, 24.48]]",2500096,Others on the ball,1639,2H,1633.927077,72,251668198,399564.0,right,LasseSorensen


In [11]:
df_defence_footed.rename(columns={'R-CB':'R_CB',"L-CB":'L_CB'},inplace=True)

**Rounding position coordinates to 2 decimal points**

In [12]:
def roundcoords(x):
    final_roundedcoords = list()
    for l in x:
        roundedcoords = [np.round(val,decimals=2) for val in l]
        final_roundedcoords.append(roundedcoords)
    return final_roundedcoords
    
df_events['positions'] = df_events['positions'].apply(lambda x: roundcoords(x))

In [10]:
df_events_roles

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,wyId,foot,playerName,role
0,8,Simple pass,[Accurate],25413,"[[50.96, 34.68], [32.24, 14.96]]",2499719,Pass,1609,1H,2.758649,85,177959171,25413.0,right,AlexandreLacazette,FWD
1,1,Air duel,"[Lost, Not accurate]",25413,"[[73.84, 31.28], [52.0, 27.880000000000003]]",2499719,Duel,1609,1H,22.551816,10,177959181,25413.0,right,AlexandreLacazette,FWD
2,10,Shot,"[Goal, Right foot, Opportunity, Position: Goal...",25413,"[[91.52, 40.120000000000005], [-0.0, 68.0]]",2499719,Shot,1609,1H,94.595788,100,177959212,25413.0,right,AlexandreLacazette,FWD
3,8,Head pass,[Not accurate],25413,"[[73.84, 20.4], [73.84, 25.159999999999997]]",2499719,Pass,1609,1H,397.881307,82,177959276,25413.0,right,AlexandreLacazette,FWD
4,1,Ground defending duel,"[Take on left, Lost, Not accurate]",25413,"[[63.44, 24.479999999999997], [53.04, 8.840000...",2499719,Duel,1609,1H,494.461238,12,177959303,25413.0,right,AlexandreLacazette,FWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595114,1,Ground attacking duel,"[Won, Accurate]",399564,"[[59.28, 9.519999999999996], [56.16, 3.4000000...",2500096,Duel,1639,2H,1275.376288,11,251668153,399564.0,right,LasseSørensen,MID
595115,8,Simple pass,[Not accurate],399564,"[[56.16, 3.4000000000000057], [48.88, 0.0]]",2500096,Pass,1639,2H,1276.749933,85,251668154,399564.0,right,LasseSørensen,MID
595116,8,Simple pass,[Accurate],399564,"[[76.96000000000001, 25.159999999999997], [61....",2500096,Pass,1639,2H,1519.322589,85,251668174,399564.0,right,LasseSørensen,MID
595117,7,Touch,[Missed ball],399564,"[[73.84, 32.64], [74.88, 24.480000000000004]]",2500096,Others on the ball,1639,2H,1633.927077,72,251668198,399564.0,right,LasseSørensen,MID


**Adding player roles to the events table**

In [14]:
roles_temp = df_players['role'].values

In [15]:
roles = list()
for i in roles_temp:
    roles.append(i['code3'])
    

In [16]:
players_roles = list(zip(roles,df_players['wyId'],df_players['playerName']))

In [17]:
df_players_roles = pd.DataFrame(players_roles,columns = ['role','playerId','playerName1'])

In [18]:
df_events_roles = df_events.merge(df_players_roles, left_on = 'playerId', right_on = 'playerId'  )

In [19]:
df_events_roles.drop(['playerName1'], axis = 1, inplace = True)

In [20]:
df_events_roles.to_pickle('/Users/shreygrover/Desktop/Projects/Project-DefendersFootedness/PreliminaryAnalysis/Data/events_roles.pkl')

**Filtering out pass data for defenders and finding total passes and total accurate passes for defenders league wise**

In [6]:
df_events_pass = df_events_roles.loc[df_events_roles['subEventName'].str.contains('pass')].loc[df_events_roles['role']=='DEF']

In [7]:
league_pass_info = dict()
league_pass_info['totalpasses'] = len(df_events_pass)

In [8]:
df_events_pass['playerName'].value_counts()

NicolasHernanOtamendi          2938
CesarAzpilicuetaTanco          2588
JanVertonghen                  2297
KyleWalker                     2223
DavinsonSanchezMina            1768
HectorBellerinMoruno           1766
DejanLovren                    1668
LaurentKoscielny               1640
IgnacioMonrealEraso            1635
ShkodranMustafi                1621
VirgilvanDijk                  1614
AlfieMawson                    1604
AntonioRudiger                 1598
HarryMaguire                   1594
JoelAndreJobMatip              1550
KyleNaughton                   1528
LuisAntonioValenciaMosquera    1512
NathanAke                      1506
MathiasJattahNjieJorgensen     1465
CharlieDaniels                 1446
SimonFrancis                   1413
RyanBertrand                   1391
VincentKompany                 1380
LewisDunk                      1379
AndrewRobertson                1363
ChristopherSchindler           1291
AndreasChristensen             1275
JoeGomez                    

In [9]:
league_pass_info['totalaccuratepasses']=len(df_events_pass[df_events_pass['tags'].apply(lambda x: "Accurate" in x)])

In [10]:
league_pass_info

{'totalpasses': 130322, 'totalaccuratepasses': 112061}

**Creating clusters for footedness patterns**

In [11]:
footedness_patterns

array(['right-right-right-left', 'right-right-left-left',
       'right-right-left', 'right-right-right', 'right-left-left',
       'right-right-right-left-left', 'right-right-left-right',
       'right-right-right-right', 'right-right-right-right-left',
       'right-left-right', 'right-right-right-left-right',
       'right-right-left-left-left', 'right-left-left-left'], dtype=object)

In [12]:
df_rrrl = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-right-left']
df_rrll = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-left-left']
df_rrl = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-left']
df_rrr = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-right']
df_rll = df_defence_footed.loc[df_defence_footed['footedness']=='right-left-left']
df_rrrll = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-right-left-left']
df_rrlr = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-left-right']
df_rrrr = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-right-right']
df_rrrrl = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-right-right-left']
df_rlr = df_defence_footed.loc[df_defence_footed['footedness']=='right-left-right']
df_rrrlr = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-right-left-right']
df_rrlll = df_defence_footed.loc[df_defence_footed['footedness']=='right-right-left-left-left']
df_rlll = df_defence_footed.loc[df_defence_footed['footedness']=='right-left-left-left']

In [13]:
df_clusters = [df_rrrl,df_rrll,df_rrl,df_rrr,df_rll,df_rrrll,df_rrlr,df_rrrr,df_rrrrl,df_rlr,df_rrrlr,df_rrlll,df_rlll]

**Populating metrics for players in each match per cluster**

In [123]:
# def fuzzycheck(a,y):
#     split_y = re.findall('[A-Z][^A-Z]*',y)
#     try:
#         match_1 = SequenceMatcher(None,a,split_y[-1]).ratio()
#         match_2 = SequenceMatcher(None,a,split_y[-2]).ratio()
#         if np.mean(match_1,match_2)>=0.37:
#             return True
#         else:
#             return False
#     except:
#         match_1 = SequenceMatcher(None,a,split_y[-1]).ratio()
#         if match_1>=0.39:
#             return True
#         else:
#             return False


# def getpass(x,y):
#     pass_df = df_events_pass.loc[(df_events_pass['playerName'].apply(lambda a: fuzzycheck(a,y))) &
#                                  (df_events_pass['matchId']==int(x))]
#     numpasses = len(pass_df)
#     print(y)
#     return numpasses

# def getaccpass(x,y):
#     pass_df = df_events_pass.loc[(df_events_pass['playerName'].apply(lambda a: fuzzycheck(a,y))) &
#                                  (df_events_pass['matchId']==int(x))]
#     numaccpasses = len(pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)])
#     print(y)
#     return numaccpasses

# def getaccpasslocs(x,y):
#     pass_df = df_events_pass.loc[(df_events_pass['playerName'].apply(lambda a: fuzzycheck(a,y))) &
#                                  (df_events_pass['matchId']==int(x))]
#     accpasslocs = pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)]['positions'].tolist()
#     print(y)
#     return accpasslocs

# def getinaccpasslocs(x,y):
#     pass_df = df_events_pass.loc[(df_events_pass['playerName'].apply(lambda a: fuzzycheck(a,y))) &
#                                  (df_events_pass['matchId']==int(x))]
#     inaccpasslocs = pass_df.loc[pass_df['tags'].apply(lambda a: "Not accurate" in a)]['positions'].tolist()
#     print(y)
#     return inaccpasslocs



In [14]:
player_map = {  'RamiroFunesMori': 'JoseRamiroFunesMori',
                'KurtZouma': 'KurtHappyZouma',
                'Danilo': 'DaniloLuizdaSilva',
                'CesarAzpilicueta': 'CesarAzpilicuetaTanco',
                'EzequielSchelotto': 'MatiasEzequielSchelotto',
                'GaetanBong': 'GaetanBongSongo',
                'HectorBellerin': 'HectorBellerinMoruno',
                'AhmedHegazi': 'AhmedHegazy',
                'JamaalLascelles': 'JamalLascelles',
                'AngelRangel': 'AngelRangelZaragoza',
                'Zanka': 'MathiasJattahNjieJorgensen',
                'ChrisLwe': 'ChrisLowe',
                'EricBailly': 'EricBertrandBailly',
                'MarcosRojo': 'FaustinoMarcosAlbertoRojo',
                'CdricSoares': 'CedricRicardoAlvesSoares',
                'AngeloOgbonna': 'AngeloObinzeOgbonna',
                'HctorBellern': 'HectorBellerinMoruno',
                'DavinsonSanchez': 'DavinsonSanchezMina',
                'JavierManquillo': 'JavierManquilloGaitan',
                'TommySmith': 'TomSmith',
                'Bruno': 'BrunoSaltorGrau',
                'GatanBong': 'GaetanBongSongo',
                'NicolsOtamendi': 'NicolasHernanOtamendi',
                'CsarAzpilicueta': 'CesarAzpilicuetaTanco',
                'AntonioRdiger': 'AntonioRudiger',
                'JosHolebas': 'JoseHolebas',
                'SamusColeman': 'SeamusColeman',
                'AllanRomoNyom': 'AllanRomeoNyom',
                'NathanAk': 'NathanAke',
                'JosephGomez': 'JoeGomez',
                'AlbertoMoreno':'AlbertoMorenoPerez',
                'LuisAntonioValencia':'LuisAntonioValenciaMosquera',
                'VictorLindelf':'VictorNilssonLindelof',
                'DavinsonSnchez':'DavinsonSanchezMina',
                'NicolasOtamendi':'NicolasHernanOtamendi',
                'NachoMonreal':'IgnacioMonrealEraso',
                'FedericoFernndez':'FedericoFernandez',
                'SebastianPrdl':'SebastianProdl',
                'CedricSoares':'CedricRicardoAlvesSoares',
                'JoelMatip':'JoelAndreJobMatip',
                'MiguelBritos':'MiguelAngelBritosCabrera',
                'VictorLindelof':'VictorNilssonLindelof',
                'JamesCollins':'JamesMichaelCollins',
                'CucoMartina':'RhuendlyMartina',
                'DavidLuiz':'DavidLuizMoreiraMarinho',
                'MollaWagu':'MollaWague',
                'JrmyPied':'JeremyPied',
                'ChancelMbemba':'ChancelMbembaMangulu',
                'PabloZabaleta':'PabloJavierZabaletaGirod',
                'KikoFemenia':'FranciscoFemeniaFar',
                'CheikhouKouyat':'CheikhouKouyate',
                'KikoFemena':'FranciscoFemeniaFar',
                'JoseFonte':'JoseMigueldaRochaFonte',
                'JosFonte':'JoseMigueldaRochaFonte',
                'JesusGamez':'JesusGamezDuarte'}


In [15]:
def getmetrics(x,y):
    try:
        y = player_map[y]
    except:
        pass
    split_y = re.findall('[A-Z][^A-Z]*',y)
    try:
        pass_df = df_events_pass.loc[(df_events_pass['playerName'].str.contains(split_y[-1]))&
                                 (df_events_pass['playerName'].str.contains(split_y[-2]))&
                                 (df_events_pass['matchId']==int(x))]
    except:
        pass_df = df_events_pass.loc[(df_events_pass['playerName'].str.contains(split_y[-1]))&
                                 (df_events_pass['matchId']==int(x))]
    numpasses = len(pass_df)
    numaccpasses = len(pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)])
    numhighpasses = len(pass_df.loc[pass_df['subEventName']=='High pass'])
    numhighaccpasses = len(pass_df.loc[(pass_df['subEventName']=='High pass') & (pass_df['tags'].apply(lambda a: "Accurate" in a))])
    accpasslocs = pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)]['positions'].tolist()
    inaccpasslocs = pass_df.loc[pass_df['tags'].apply(lambda a: "Not accurate" in a)]['positions'].tolist()
    acchighpasslocs = pass_df.loc[(pass_df['subEventName']=='High pass') & (pass_df['tags'].apply(lambda a: "Accurate" in a))]['positions'].tolist()
    inacchighpasslocs = pass_df.loc[(pass_df['subEventName']=='High pass') & (pass_df['tags'].apply(lambda a: "Not accurate" in a))]['positions'].tolist()
    return [numpasses,numaccpasses,numhighpasses,numhighaccpasses,accpasslocs,inaccpasslocs,acchighpasslocs,inacchighpasslocs]


In [16]:
getmetrics(2500081,"Bruno")

[27,
 22,
 5,
 3,
 [[[30.16, 5.44], [26.0, 15.64]],
  [[33.28, 12.92], [29.12, 29.92]],
  [[32.24, 4.76], [37.44, 14.96]],
  [[75.92, 4.76], [83.2, 4.76]],
  [[69.68, 11.56], [78.0, 10.88]],
  [[32.24, 4.76], [36.4, 17.0]],
  [[78.0, 6.12], [71.76, 8.16]],
  [[32.24, 23.12], [26.0, 29.92]],
  [[47.84, 27.88], [28.08, 34.68]],
  [[4.16, 6.12], [16.64, 4.08]],
  [[71.76, 6.12], [74.88, 9.52]],
  [[39.52, 16.32], [23.92, 32.64]],
  [[43.68, 12.92], [30.16, 34.0]],
  [[35.36, 10.88], [29.12, 27.88]],
  [[46.8, 28.56], [93.6, 51.68]],
  [[9.36, 18.36], [10.4, 23.12]],
  [[63.44, 7.48], [65.52, 2.72]],
  [[64.48, 2.04], [58.24, 28.56]],
  [[17.68, 1.36], [43.68, 1.36]],
  [[31.2, 12.92], [34.32, 3.4]],
  [[58.24, 7.48], [30.16, 24.48]],
  [[36.4, 7.48], [60.32, 18.36]]],
 [[[32.24, 3.4], [53.04, 33.32]],
  [[20.8, 20.4], [40.56, 8.84]],
  [[17.68, 5.44], [59.28, 6.8]],
  [[87.36, 4.76], [76.96, 46.92]],
  [[36.4, 1.36], [60.32, 16.32]]],
 [[[46.8, 28.56], [93.6, 51.68]],
  [[17.68, 1.36], [4

In [17]:
new_cols = ['RB_all',
            'R_CB_all',
            'L_CB_all',
            'LB_all',
            'RCB_all',
            'CB_all',
            'LCB_all',
            'RWB_all',
            'LWB_all']

In [18]:
#R_CB - Right center back for 4 defender formation
#RCB - Right center back for 3 or 5 defender formation
#L_CB - Left center back for 4 defender formation
#LCB - Left center back for 3 or 5 defender formation
df_clusters_updated = list()
for df in tqdm(df_clusters):
    df = df.reindex(columns = df.columns.tolist() + new_cols)
    if df.iloc[0]['backline'] == 4.0:     
        df['RB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RB), axis=1)
        df['R_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['R-CB']), axis=1)
        df['L_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['L-CB']), axis=1)
        df['LB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LB), axis=1)
        df_clusters_updated.append(df)
    
    elif df.iloc[0]['backline'] == 3.0:
        print("hi")
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df_clusters_updated.append(df)
        
    elif df.iloc[0]['backline'] == 5.0:
        print("hi")
        df['RWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RWB), axis=1)
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df['LWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LWB), axis=1)
        df_clusters_updated.append(df)

 15%|████████████▌                                                                     | 2/13 [04:48<29:49, 162.67s/it]

hi


 23%|██████████████████▉                                                               | 3/13 [05:21<20:36, 123.66s/it]

hi


 31%|█████████████████████████▌                                                         | 4/13 [05:59<14:42, 98.02s/it]

...

 77%|███████████████████████████████████████████████████████████████                   | 10/13 [07:18<01:05, 21.73s/it]

hi


 85%|█████████████████████████████████████████████████████████████████████▍            | 11/13 [07:18<00:30, 15.45s/it]

hi


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [07:21<00:00, 33.93s/it]


In [ ]:
df_clusters = [df_rrrl,df_rrll,df_rrl,df_rrr,df_rll,df_rrrll,df_rrlr,df_rrrr,df_rrrrl,df_rlr,df_rrrlr,df_rrlll,df_rlll]

In [215]:
df_players.loc[df_players['playerName'].str.contains("DavidLuizMoreiraMarinho")]

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId,playerName
2896,"{'name': 'Portugal', 'id': 620, 'alpha3code': ...",84,David Luiz,,Moreira Marinho,1610,1987-04-22,189,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Brazil', 'id': 76, 'alpha3code': 'BR...",7892,right,David Luiz,null,DavidLuizMoreiraMarinho


In [237]:
df_clusters_updated[6].loc[df_clusters_updated[6]['RB_all'].apply(lambda p : p[0]==0)]

,wyId,team,team_defense,RB,R_CB,L_CB,LB,RCB,CB,LCB,RWB,LWB,backline,match,gameweek,teamsData,dateutc,venue,referees,score,footedness,RB_all,R_CB_all,L_CB_all,LB_all,RCB_all,CB_all,LCB_all,RWB_all,LWB_all


In [19]:
df_clusters_metrics = list()
for df in tqdm(df_clusters_updated):
    if df.iloc[0]['backline'] == 4.0:
        df[['RB_pass','RB_accpass','RB_highpass','RB_acchighpass','RB_accpassloc','RB_inaccpassloc','RB_acchighpassloc','RB_inacchighpassloc']] = pd.DataFrame(df['RB_all'].to_list(), index=df.index)
        df[['R_CB_pass','R_CB_accpass','R_CB_highpass','R_CB_acchighpass','R_CB_accpassloc','R_CB_inaccpassloc','R_CB_acchighpassloc','R_CB_inacchighpassloc']] = pd.DataFrame(df['R_CB_all'].to_list(), index=df.index)
        df[['L_CB_pass','L_CB_accpass','L_CB_highpass','L_CB_acchighpass','L_CB_accpassloc','L_CB_inaccpassloc','L_CB_acchighpassloc','L_CB_inacchighpassloc']] = pd.DataFrame(df['L_CB_all'].to_list(), index=df.index)
        df[['LB_pass','LB_accpass','LB_highpass','LB_acchighpass','LB_accpassloc','LB_inaccpassloc','LB_acchighpassloc','LB_inacchighpassloc']] = pd.DataFrame(df['LB_all'].to_list(), index=df.index)
        df.drop(['RB_all','R_CB_all','L_CB_all','LB_all','RCB_all','LCB_all','CB_all','RWB_all','LWB_all'], axis=1, inplace = True)
        df_clusters_metrics.append(df)
       
    elif df.iloc[0]['backline'] == 3.0:
        df[['RCB_pass','RCB_accpass','RCB_highpass','RCB_acchighpass','RCB_accpassloc','RCB_inaccpassloc','RCB_acchighpassloc','RCB_inacchighpassloc']] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[['CB_pass','CB_accpass','CB_highpass','CB_acchighpass','CB_accpassloc','CB_inaccpassloc','CB_acchighpassloc','CB_inacchighpassloc']] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[['LCB_pass','LCB_accpass','LCB_highpass','LCB_acchighpass','LCB_accpassloc','LCB_inaccpassloc','LCB_acchighpassloc','LCB_inacchighpassloc']] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df.drop(['RB_all','R_CB_all','L_CB_all','LB_all','RCB_all','LCB_all','CB_all','RWB_all','LWB_all'], axis=1, inplace = True)
        df_clusters_metrics.append(df)
       
    elif df.iloc[0]['backline'] == 5.0:
        df[['RCB_pass','RCB_accpass','RCB_highpass','RCB_acchighpass','RCB_accpassloc','RCB_inaccpassloc','RCB_acchighpassloc','RCB_inacchighpassloc']] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[['CB_pass','CB_accpass','CB_highpass','CB_acchighpass','CB_accpassloc','CB_inaccpassloc','CB_acchighpassloc','CB_inacchighpassloc']] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[['LCB_pass','LCB_accpass','LCB_highpass','LCB_acchighpass','LCB_accpassloc','LCB_inaccpassloc','LCB_acchighpassloc','LCB_inacchighpassloc']] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df[['RWB_pass','RWB_accpass','RWB_highpass','RWB_acchighpass','RWB_accpassloc','RWB_inaccpassloc','RWB_acchighpassloc','RWB_inacchighpassloc']] = pd.DataFrame(df['RWB_all'].to_list(), index=df.index)
        df[['LWB_pass','LWB_accpass','LWB_highpass','LWB_acchighpass','LWB_accpassloc','LWB_inaccpassloc','LWB_acchighpassloc','LWB_inacchighpassloc']] = pd.DataFrame(df['LWB_all'].to_list(), index=df.index)
        df.drop(['RB_all','R_CB_all','L_CB_all','LB_all','RCB_all','LCB_all','CB_all','RWB_all','LWB_all'], axis=1, inplace = True)
        df_clusters_metrics.append(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 36.60it/s]


In [20]:
names = ['rrrl','rrll','rrl','rrr','rll','rrrll','rrlr','rrrr','rrrrl','rlr','rrrlr','rrlll','rlll']

for i,df in enumerate(df_clusters_metrics):
    df.to_pickle(f'C:/Users/Anuj/Desktop/Work/Football/data/clusters/clusters_v3/cluster_{names[i]}.pkl')

In [250]:
df_events_pass.loc[(df_events_pass['matchId']==2500065) & (df_events_pass['playerName'].str.contains('FedericoFernandez'))]

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,wyId,foot,playerName,role
309752,8,Head pass,"[Interception, Accurate]",21164,"[[93.6, 34.0], [95.68, 17.68]]",2500065,Pass,10531,1H,109.904906,82,245422246,21164.0,right,FedericoFernandez,DEF
309753,8,High pass,[Accurate],21164,"[[86.32, 32.64], [59.28, 35.36]]",2500065,Pass,10531,1H,358.156361,83,245422308,21164.0,right,FedericoFernandez,DEF
309754,8,Head pass,"[Interception, Not accurate]",21164,"[[80.08, 28.56], [81.12, 26.52]]",2500065,Pass,10531,1H,422.402779,82,245422333,21164.0,right,FedericoFernandez,DEF
309757,8,High pass,[Not accurate],21164,"[[74.88, 32.64], [33.28, 4.76]]",2500065,Pass,10531,1H,739.257750,83,245422436,21164.0,right,FedericoFernandez,DEF
309759,8,Simple pass,[Accurate],21164,"[[82.16, 37.4], [78.0, 35.36]]",2500065,Pass,10531,1H,2105.179456,85,245422806,21164.0,right,FedericoFernandez,DEF
309761,8,Simple pass,[Accurate],21164,"[[29.12, 34.0], [31.2, 17.68]]",2500065,Pass,10531,2H,2.450625,85,245423018,21164.0,right,FedericoFernandez,DEF
309764,8,Simple pass,[Accurate],21164,"[[12.48, 32.64], [12.48, 56.44]]",2500065,Pass,10531,2H,199.976418,85,245423083,21164.0,right,FedericoFernandez,DEF
